# Train prediction models for texture 
* convolutional neural network over single samples; input features_SA and features RA of size (n_pins, n_pins) representing firing rate on an n_pins x n_pins square array                
* training data concatenated over speed leaving one speed out but retained for testing
* therefore total of n_speed models (here 10), one per speed
* i.e. prediction of texture independent of held-out speed

To run, first edit dir_data to path where data is stored; spaital/process_static in this directory should be run first.

Because of the large amount of data, a temporary folder is used for the training data, so it can be deleted after training the static model. 

In [1]:
import os
import pickle

def open_obj(name):
    with open(name + '.pkl', 'rb') as f:
        obj = pickle.load(f)
    return obj

In [2]:
dir_data = os.environ["DATAPATH"] + r"/open/afferents-tactile-textures-jrsi2022"
dir_temp = os.environ["TEMPPATH"] + r"/static"

n_speeds = 10
n_textures = 13
n_pins = 19

In [3]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session, clear_session
from keras import optimizers, regularizers, callbacks, Input, Model
from keras.models import Sequential, load_model
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout, BatchNormalization
from numpy.random import seed
from tensorflow import set_random_seed

C:\Users\Nathan\Anaconda3\envs\tactip1\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Nathan\Anaconda3\envs\tactip1\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Nathan\Anaconda3\envs\tactip1\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Nathan\Anaconda3\envs\tactip1\lib\site-packages\tensorf

In [4]:
def define_model(X_train, y_train, X_val, y_val, es, cp):
    
    clear_session()
    
    config = tf.ConfigProto(gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.8)) # device_count = {'GPU': 1}
    config.gpu_options.allow_growth = True
    session = tf.Session(config=config)
    set_session(session)
    
    seed(1)
    set_random_seed(2)
    
    model = Sequential()
    model.add(Conv2D(64, (3,3), activation='relu', padding="same", input_shape=(n_pins,n_pins,1)))
    model.add(MaxPooling2D((2,2)))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3,3), activation='relu', padding="same"))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3,3), activation='relu', padding="valid"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2,2)))
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(32, kernel_regularizer=regularizers.l2(0.005), activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    model.add(Dense(16, kernel_regularizer=regularizers.l2(0.005), activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))    
    model.add(Dense(n_textures, kernel_regularizer=regularizers.l2(0.005), activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer = optimizers.Adam(lr=1e-3), metrics=['accuracy'])
    model.fit(X_train, y_train, validation_data=(X_val,y_val), epochs=150, batch_size=64, shuffle=True, callbacks = [es,cp])

# leave-one-speed-out

## SA

In [5]:
for i in range(n_speeds):
    data_set = dir_temp + rf"/{i}"
    X_train = open_obj(data_set + r"/X_train_SA")
    X_val = open_obj(data_set + r"/X_val_SA")
    y_train = open_obj(data_set + r"/y_train_SA")
    y_val = open_obj(data_set + r"/y_val_SA")
    
    dir_model = dir_data + rf"/models/static_SA/{i}"
    os.makedirs(dir_model, exist_ok=True)
    cp = callbacks.ModelCheckpoint(dir_model + r"/model_{epoch:02d}_{val_accuracy:.2f}.hdf5", monitor="val_accuracy", save_best_only=True)
    es = callbacks.EarlyStopping(monitor="val_accuracy", patience=20, restore_best_weights=True)
    define_model(X_train, y_train[1], X_val, y_val[1], es, cp)



Train on 585000 samples, validate on 32500 samples
Epoch 1/150
585000/585000 [==============================] - 94s 160us/step - loss: 0.4886 - accuracy: 0.9124 - val_loss: 3.0090 - val_accuracy: 0.3709
Epoch 2/150
585000/585000 [==============================] - 92s 157us/step - loss: 0.1724 - accuracy: 0.9833 - val_loss: 3.2361 - val_accuracy: 0.3904
Epoch 3/150
585000/585000 [==============================] - 91s 155us/step - loss: 0.1444 - accuracy: 0.9866 - val_loss: 3.2524 - val_accuracy: 0.4041
Epoch 4/150
585000/585000 [==============================] - 91s 156us/step - loss: 0.1264 - accuracy: 0.9888 - val_loss: 3.2190 - val_accuracy: 0.4020
Epoch 5/150
585000/585000 [==============================] - 91s 155us/step - loss: 0.1156 - accuracy: 0.9903 - val_loss: 3.4453 - val_accuracy: 0.3921
Epoch 6/150
585000/585000 [==============================] - 91s 155us/step - loss: 0.1084 - accuracy: 0.9910 - val_loss: 3.7716 - val_accuracy: 0.3849
Epoch 7/150
585000/585000 [========

## RA

In [6]:
for i in range(n_speeds):
    data_set = dir_temp + rf"/{i}"
    X_train = open_obj(data_set + rf"/X_train_RA")
    X_val = open_obj(data_set + rf"/X_val_RA")
    y_train = open_obj(data_set + rf"/y_train_RA")
    y_val = open_obj(data_set + rf"/y_val_RA")
    
    dir_model = dir_data + rf"/models/static_RA/{i}"
    os.makedirs(dir_model, exist_ok=True)
    cp = callbacks.ModelCheckpoint(dir_model + r"/model_{epoch:02d}_{val_accuracy:.2f}.hdf5", monitor="val_accuracy", save_best_only=True)
    es = callbacks.EarlyStopping(monitor="val_accuracy", patience=20, restore_best_weights=True)
    define_model(X_train, y_train[1], X_val, y_val[1], es, cp)

Train on 585000 samples, validate on 32500 samples
Epoch 1/150
585000/585000 [==============================] - 87s 149us/step - loss: 1.9502 - accuracy: 0.4037 - val_loss: 2.4310 - val_accuracy: 0.2411
Epoch 2/150
585000/585000 [==============================] - 84s 144us/step - loss: 1.7227 - accuracy: 0.4766 - val_loss: 2.3793 - val_accuracy: 0.2486
Epoch 3/150
585000/585000 [==============================] - 84s 144us/step - loss: 1.6671 - accuracy: 0.4946 - val_loss: 2.3944 - val_accuracy: 0.2468
Epoch 4/150
585000/585000 [==============================] - 84s 144us/step - loss: 1.6285 - accuracy: 0.5079 - val_loss: 2.3570 - val_accuracy: 0.2598
Epoch 5/150
585000/585000 [==============================] - 84s 144us/step - loss: 1.6009 - accuracy: 0.5163 - val_loss: 2.4954 - val_accuracy: 0.2382
Epoch 6/150
585000/585000 [==============================] - 84s 144us/step - loss: 1.5815 - accuracy: 0.5236 - val_loss: 2.4441 - val_accuracy: 0.2518
Epoch 7/150
585000/585000 [==========